# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

# Data Load

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('testA.csv')

# Feature Engineering

In [3]:
train_data['datetime'] = pd.to_datetime(train_data['datetime'])
test_data['datetime'] = pd.to_datetime(test_data['datetime'])
train_data['datetime_hour'] = train_data['datetime'].dt.hour
test_data['datetime_hour'] = test_data['datetime'].dt.hour

train_data.drop('datetime', axis=1, inplace=True)
test_data.drop('datetime', axis=1, inplace=True)



for col in ['ad_id', 'ad_set_id', 'campaign_id', 'product_id', 'account_id', 'post_id_emb', 'post_type', 'countries']:
    lbl = LabelEncoder()
    lbl.fit(list(train_data[col]) + list(test_data[col]))
    train_data[col] = lbl.transform(list(train_data[col]))
    test_data[col] = lbl.transform(list(test_data[col]))



train_data['product_id_roi_mean'] = train_data['product_id'].map(train_data.groupby(['product_id'])['roi'].mean())
test_data['product_id_roi_mean'] = test_data['product_id'].map(train_data.groupby(['product_id'])['roi'].mean())

train_data['account_id_roi_mean'] = train_data['account_id'].map(train_data.groupby(['account_id'])['roi'].mean())
test_data['account_id_roi_mean'] = test_data['account_id'].map(train_data.groupby(['account_id'])['roi'].mean())

train_data['countries_roi_mean'] = train_data['countries'].map(train_data.groupby(['countries'])['roi'].mean())
test_data['countries_roi_mean'] = test_data['countries'].map(train_data.groupby(['countries'])['roi'].mean())

train_data['datetime_hour_roi_mean'] = train_data['datetime_hour'].map(train_data.groupby(['datetime_hour'])['roi'].mean())
test_data['datetime_hour_roi_mean'] = test_data['datetime_hour'].map(train_data.groupby(['datetime_hour'])['roi'].mean())

In [5]:
train_data

,ad_id,ad_set_id,campaign_id,product_id,account_id,post_id_emb,post_type,countries,gender,age_min,...,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi,datetime_hour,product_id_roi_mean,account_id_roi_mean,countries_roi_mean,datetime_hour_roi_mean
0,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,0,1.606082,2.249979,1.563869,1.805425
1,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,1,1.606082,2.249979,1.563869,1.798679
2,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,2,1.606082,2.249979,1.563869,1.802189
3,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,3,1.606082,2.249979,1.563869,1.802148
4,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,4,1.606082,2.249979,1.563869,1.801525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656414,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,8,1.549928,1.502077,1.433427,1.829020
656415,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,9,1.549928,1.502077,1.433427,1.817643
656416,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,10,1.549928,1.502077,1.433427,1.809741
656417,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,11,1.549928,1.502077,1.433427,1.803130


# Model

In [ ]:
# lightgbm
from lightgbm import LGBMRegressor
model = LGBMRegressor()

model.fit(
    train_data.iloc[:].drop('roi', axis=1),
    train_data.iloc[:]['roi'], categorical_feature=['ad_id', 'ad_set_id', 'campaign_id', 'product_id', 'account_id', 'post_id_emb', 'post_type', 'countries']
)

df = pd.read_csv('提交示例.csv')
df['roi'] = model.predict(test_data.iloc[:].drop('uuid', axis=1))
df.to_csv('submit.csv', index=None)

In [ ]:
# Time Series

In [ ]:
# Linear Regression